In [1]:
#import dependencies 
import pandas as pd
from path import Path
from sklearn.naive_bayes import MultinomialNB, CategoricalNB
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [2]:
#Take in data
file_path = Path("./Resources/cbp_data.csv")
cbp_df = pd.read_csv(file_path)
cbp_df.head()

/Users/ebonybrown/opt/anaconda3/envs/PythonData/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (2,7,8,9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,date_in,date_out,app_date,hours_in_custody,age_group,gender,citizenship,border,sector,field_office,source
0,2017-01-20 00:10:00,2017-01-20 10:08:00,NaN,9.950000,6-8 years,Female,EL SALVADOR,SBO,(b)(7)(E ),NaN,BP
1,2017-01-20 00:15:00,2017-01-24 17:30:00,NaN,113.233333,3-5 years,Female,GUATEMALA,SBO,(b)(7)(E ),NaN,BP
2,2017-01-20 00:22:00,2017-01-24 17:47:00,NaN,113.416667,3-5 years,Female,BRAZIL,SBO,(b)(7)(E ),NaN,BP
3,2017-01-20 00:30:00,2017-01-21 06:35:00,NaN,30.083333,12-14 years,Male,EL SALVADOR,SBO,(b)(7)(E ),NaN,BP
4,2017-01-20 00:30:00,2017-01-21 13:03:00,NaN,36.533333,3-5 years,Male,HONDURAS,SBO,(b)(7)(E ),NaN,BP


In [3]:
#drop columns and null values
cpb_df_update = cbp_df.drop(["date_in", "date_out", "border", "sector", "field_office", "source", "app_date"], axis=1).dropna()

In [4]:
country_num = {
    'EL SALVADOR': 1, 
    'GUATEMALA': 2, 
    'BRAZIL': 4, 
    'HONDURAS': 5, 
    'INDIA': 6,
    'MEXICO': 7, 
    'PERU': 8, 
    'NICARAGUA': 9, 
    'ROMANIA': 10, 
    'BANGLADESH': 11, 
    'ECUADOR': 12,
    'UKRAINE': 13, 
    'COLOMBIA': 14, 
    'BELIZE': 15, 
    'YEMEN': 16, 
    'VENEZUELA': 17,
    'UNITED KINGDOM': 18, 
    'CHILE': 19, 
    'SUDAN': 20, 
    'CANADA': 21, 
    'NEPAL': 22,
    'CHINA, PEOPLES REPUBLIC OF': 23, 
    'HAITI': 24, 
    'BAHAMAS': 25, 
    'NIGERIA': 26,
    'CUBA': 27, 
    'SOUTH AFRICA': 28, 
    'SOMALIA': 29, 
    'DOMINICAN REPUBLIC': 30, 
    'POLAND': 31,
    'ARGENTINA': 32, 
    'COSTA RICA': 33, 
    'VIETNAM': 34, 
    'PANAMA': 35, 
    'JAMAICA': 36,
    'PAKISTAN': 37, 
    'IRELAND': 38, 
    'PORTUGAL': 39, 
    'FRANCE': 40, 
    'SRI LANKA': 41, 
    'GAMBIA': 42,
    'ALBANIA': 43, 
    'SIERRA LEONE': 44, 
    'ERITREA': 45, 
    'MALAYSIA': 46, 
    'THAILAND': 47,
    'SPAIN': 48, 
    'BOLIVIA': 49, 
    'HUNGARY': 50, 
    'AFGHANISTAN': 51, 
    'BURMA': 52, 
    'GHANA': 53,
    'ITALY': 54, 
    'GUYANA': 55, 
    'IRAN': 56, 
    'RUSSIA': 57, 
    'KAZAKHSTAN': 58, 
    'JORDAN': 59,
    'CONGO': 60, 
    'GUADELOUPE': 61, 
    'TRINIDAD AND TOBAGO': 62, 
    'TURKEY': 63, 
    'GUINEA': 64,
    'UZBEKISTAN': 65, 
    'QATAR': 66, 
    'URUGUAY': 67, 
    'TAJIKISTAN': 68,
    'DEM REP OF THE CONGO': 69, 
    'ANGOLA': 70, 
    'TOGO': 71, 
    'CAMEROON': 72,
    'IVORY COAST': 73, 
    'BELARUS': 74, 
    'SWITZERLAND': 75, 
    'BARBADOS': 76, 
    'PHILIPPINES': 77,
    'BELGIUM': 78, 
    'KOSOVO': 79, 
    'KENYA': 80, 
    'BENIN': 81, 
    'FRENCH GUIANA': 82, 
    'ARMENIA': 83,
    'EGYPT': 84, 
    'REPUBLIC OF CONGO (BRAZZAVILLE)': 85,
    'DEMOCRATIC REPUBLIC OF CONGO (ZAIRE)': 86, 
    'MOLDOVA': 87, 
    'SYRIA': 88,
    'CHINA (MAINLAND)': 23, 
    'ETHIOPIA': 89, 
    'KYRGYZSTAN': 90, 
    'UNKNOWN': 0, 
    'UGANDA': 91,
    'RWANDA': 92, 
    'AZERBAIJAN': 93, 
    'GEORGIA': 94, 
    'GUAM': 95,
    'UNITED STATES OF AMERICA': 96, 
    'GERMANY': 97, 
    'TURKMENISTAN': 98,
    'EQUATORIAL GUINEA': 99, 
    'SAUDI ARABIA': 100, 
    'MALI': 101, 
    'IRAQ': 102, 
    'SOUTH KOREA': 103,
    'PARAGUAY': 104, 
    'REPUBLIC OF SOUTH AFRICA': 105, 
    'UNITED ARAB EMIRATES': 106,
    'BURKINA FASO (UPPER VOLTA)': 107, 
    'LIBYA': 108, 
    'AUSTRALIA': 109, 
    'SLOVAKIA': 110
}

In [5]:
cpb_df_update["country_code"] = cpb_df_update["citizenship"].apply(lambda x: country_num[x])

In [6]:
cpb_df_new = cpb_df_update.drop(["citizenship"], axis=1)
cpb_df_new.head()

,hours_in_custody,age_group,gender,country_code
0,9.950000,6-8 years,Female,1
1,113.233333,3-5 years,Female,2
2,113.416667,3-5 years,Female,4
3,30.083333,12-14 years,Male,1
4,36.533333,3-5 years,Male,5


In [7]:
#Encode data
cbp_df_encoded = pd.get_dummies(cpb_df_new, columns=["age_group", "gender"])
cbp_df_encoded

,hours_in_custody,country_code,age_group_1-2 years,age_group_12-14 years,age_group_15-18 years,age_group_3-5 years,age_group_6-8 years,age_group_9-11 years,age_group_Under 1 year,gender_F - FEMALE,gender_Female,gender_M - MALE,gender_Male,gender_U - UNKNOWN
0,9.950000,1,0,0,0,0,1,0,0,0,1,0,0,0
1,113.233333,2,0,0,0,1,0,0,0,0,1,0,0,0
2,113.416667,4,0,0,0,1,0,0,0,0,1,0,0,0
3,30.083333,1,0,1,0,0,0,0,0,0,0,0,1,0
4,36.533333,5,0,0,0,1,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
583803,4.770000,7,0,0,1,0,0,0,0,0,0,1,0,0
583804,21.620000,7,0,0,1,0,0,0,0,0,0,1,0,0
583805,2.810000,7,0,1,0,0,0,0,0,1,0,0,0,0
583806,20.760000,7,0,0,1,0,0,0,0,1,0,0,0,0


In [8]:
#scale and nomalize data
data_scaler = StandardScaler()
cbp_df_scaled = data_scaler.fit_transform(cbp_df_encoded)